In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
import statsmodels.api as smi
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

In [31]:
def create_single_game_dimension(og_df):
    """Create a DataFrame where each row is a single game, and each country's viewership is a column
    Will do this by adding new column for a Game_ID and taking transpose at the end"""
    all_countries = ["C-%s" % e for e in og_df["Country"].unique().tolist()]
    df_index = ["Season", "Home_Team", "Away_Team", "Game_Date"] + all_countries
    new_df = pd.DataFrame(index=df_index)
    for game_id in og_df["Game_ID"].unique():
        # Define the new series using the same index as the dataframe it will be concatenated to
        game_series = pd.Series(index=df_index)
        # Isolate the rows relevant for the current game
        game_rows = og_df.loc[og_df["Game_ID"] == game_id, :]

        # Since much of the data is repeated, get important information from the first row of the game
        game_series.loc[["Season", "Home_Team", "Away_Team", "Game_Date"]] = game_rows.iloc[0, :].loc[["Season", "Home_Team", "Away_Team", "Game_Date"]]

        # Add each country's viewership to the new series in the appropriate index location
        # NOTE: There is probably a more efficient way to do this
        for i,row in game_rows.iterrows():
            game_series.loc["C-%s" % row["Country"]] = row["Rounded Viewers"]

        # Add the game's data as a new column in the full df
        new_df[game_id] = game_series

    # Currently the columns are game_ids and the rows are the information/data,
    # swap these to be more compatible with supervised learning methods using transpose
    new_df = new_df.T

    # Add the total viewers by summing over the country columns
    country_cols = [e for e in new_df.columns if "C-" in e]
    new_df["TotalViewers"] = new_df.loc[:, country_cols].sum(axis=1)
    return new_df

In [32]:
def create_team_vector(single_game_df):
    """Function for creating a two-hot vector with bits representing the presence of specific teams in the game"""
    # Converting teams to integer encoding
    team_encoder = LabelEncoder()
    team_encoder.fit(np.ravel([single_game_df["Home_Team"], single_game_df["Away_Team"]]))
    single_game_df["Home_Team_Code"] = team_encoder.transform(single_game_df["Home_Team"])
    single_game_df["Away_Team_Code"] = team_encoder.transform(single_game_df["Away_Team"])

    vector_encoder = OneHotEncoder(sparse=False)
    away_vectors = vector_encoder.fit_transform(single_game_df["Away_Team_Code"].values.reshape(-1, 1))
    home_vectors = vector_encoder.fit_transform(single_game_df["Home_Team_Code"].values.reshape(-1, 1))
    combined_vector = away_vectors + home_vectors
    combined_df = pd.DataFrame(data=combined_vector, index=single_game_df.index,
                               columns=["T-%s" % e for e in team_encoder.classes_])

    merged_df = pd.concat([single_game_df, combined_df], axis=1)
    return merged_df

In [6]:
# Original training set given by NBA
training_filename = "training_set.csv"
original_df = pd.read_csv(training_filename)
# full_df = create_single_game_dimension(og_df=original_df)
# # # there are some N/A values for numeric dtypes - we replace them with n/a
# full_df = full_df.fillna(0)
# # # Intermediate saving since create_single_game_df takes a few seconds to run
# full_df.to_csv("training_set_games.csv", index=True)

In [34]:
team_df = create_team_vector(full_df)

# Adding day of week and indicator if that day is a weeknight (or not)
team_df["DayOfWeek"] = pd.to_datetime(team_df["Game_Date"]).dt
team_df["Weeknight"] = team_df["DayOfWeek"].apply(lambda cell: 1 if cell in ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday"] else 0)

# Outputting the revised data frame in a 'per game' format
team_df.to_csv("training_set_games.csv", index=True, index_label="Game_ID")

In [35]:
# Read in file created from 'single_game_reformatting.py' script
input_filename = "training_set_games.csv"
df = pd.read_csv(input_filename)

# Get the columns to be used as inputs (Team indicator columns)
team_cols = [e for e in df.columns if "T-" in e]
input_cols = team_cols + ["Weeknight"]

# Get data in supervised learning format f:X -> y
X = df.loc[:, input_cols]
y = df.loc[:, "TotalViewers"]

# Initialize dictionary for tracking accuracy measures, where Mean- is using the training set mean for predictions
accuracy_dict = {"MSE":[],
                 "Mean-MSE":[],
                 "MAPE":[],
                 "Mean-MAPE":[]}

In [36]:
# Perform 10-Fold Cross-Validation
kfold = KFold(n_splits=100, shuffle=True)
for train_indices, test_indices in kfold.split(X):
    # Get the training data and fit a LinearRegression to the training data
    X_train, y_train = X.loc[train_indices, :], y.loc[train_indices]
    fold_model = LinearRegression()
    fold_model.fit(X=X_train, y=y_train)

    # Make predictions on the test data from the trained model
    X_test, y_test = X.loc[test_indices, :], y.loc[test_indices]
    predictions = fold_model.predict(X=X_test)

    # Calculate the Mean Square Error of the current test fold predictions
    error = predictions - y_test
    mean_error = y_train.mean() - y_test

    fold_mse = np.mean(error**2)
    accuracy_dict["MSE"].append(fold_mse)

    # Calculate the Sum of Squared Error of the current test fold predictions
    fold_sse = np.mean(mean_error**2)
    accuracy_dict["Mean-MSE"].append(fold_sse)

    # Calculate the Mean Absolute Percentage Error (MAPE) used by NBA to grade submissions
    fold_mape = np.mean(np.abs(error)/y_test)
    accuracy_dict["MAPE"].append(fold_mape)

    fold_mean_mape = np.mean(np.abs(mean_error)/y_test)
    accuracy_dict["Mean-MAPE"].append(fold_mean_mape)
# Converting the accuracy dictionary to a DataFrame
#   with columns as the accuracy measures and index as the fold number
results_df = pd.DataFrame.from_dict(accuracy_dict, orient='columns')
results_df["R2"] = results_df.apply(lambda row: 1.0 - row["MSE"]/row["Mean-MSE"], axis=1)
results_df["MAPE-R2"] = results_df.apply(lambda row: 1.0 - row["MAPE"]/row["Mean-MAPE"], axis=1)
print(results_df)
print(results_df.mean(axis=0).round(3))

# Using Statsmodels for output on full set to see if the coefficients make sense
# (they appear to, as CLE and GSW are highest, MEM and PHX are lowest)
#    Having weeknight games also appears to drop the amount of viewers
# The explanatory R-Squared is 0.603 with Adj R-Squared of 0.597,
#   the closeness of these to the CV-R2 indicates there is not much overfitting occurring here
Xc = smi.add_constant(X)
ols = smi.OLS(endog=y, exog=Xc)
results = ols.fit()
print(results.summary())

        MAPE           MSE  Mean-MAPE      Mean-MSE        R2   MAPE-R2
0   0.485971  5.382977e+07   0.637226  4.171100e+07 -0.290541  0.237364
1   0.525961  3.319694e+07   0.934864  7.978836e+07  0.583938  0.437393
2   0.405278  3.282608e+07   0.775078  7.020081e+07  0.532398  0.477113
3   0.422529  3.666106e+07   0.696466  5.951821e+07  0.384036  0.393324
4   0.315235  4.776609e+07   0.468029  1.114675e+08  0.571480  0.326463
5   0.345960  1.950694e+07   0.668323  7.657090e+07  0.745243  0.482346
6   0.450924  3.174249e+07   0.692554  5.503357e+07  0.423216  0.348898
7   0.353485  3.294375e+07   0.768147  6.154686e+07  0.464737  0.539821
8   0.303738  3.194189e+07   0.574921  1.156490e+08  0.723803  0.471687
9   0.331718  6.156154e+07   0.522929  1.265663e+08  0.513602  0.365653
10  0.712497  4.234233e+07   1.133115  7.024997e+07  0.397262  0.371205
11  0.430532  1.311851e+08   0.825728  2.640366e+08  0.503156  0.478603
12  0.268812  2.487284e+07   0.532754  4.354248e+07  0.428768  0

/home/sahluwalia/anaconda3/lib/python3.5/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/home/sahluwalia/anaconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/sahluwalia/anaconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/sahluwalia/anaconda3/lib/python3.5/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [37]:
results_df.head()

,MAPE,MSE,Mean-MAPE,Mean-MSE,R2,MAPE-R2
0,0.485971,53829767.35,0.637226,4.171100e+07,-0.290541,0.237364
1,0.525961,33196938.90,0.934864,7.978836e+07,0.583938,0.437393
2,0.405278,32826075.55,0.775078,7.020081e+07,0.532398,0.477113
3,0.422529,36661056.85,0.696466,5.951821e+07,0.384036,0.393324
4,0.315235,47766086.05,0.468029,1.114675e+08,0.571480,0.326463


In [3]:
# incorporating both game and team-specific features

games_df = pd.read_csv('game_data.csv')

In [4]:
games_df.head()

,Season,Game_ID,Game_Date,Team,Location,Wins_Entering_Gm,Losses_Entering_Gm,Team_Minutes,Final_Score,Lead_Changes,Ties,Largest_Lead,Full_Timeouts,Short_Timeouts,Qtr_4_Score,L2M_Score
0,2016-17,21600001,10/25/2016,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,1.0,82.0,113.0
1,2016-17,21600001,10/25/2016,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,2.0,64.0,84.0
2,2016-17,21600002,10/25/2016,POR,H,0,0,240.0,113.0,6.0,6.0,13.0,5.0,0.0,77.0,104.0
3,2016-17,21600002,10/25/2016,UTA,A,0,0,240.0,104.0,5.0,1.0,8.0,5.0,2.0,83.0,102.0
4,2016-17,21600003,10/25/2016,GSW,H,0,0,240.0,100.0,1.0,2.0,4.0,3.0,1.0,77.0,100.0


In [7]:
combined = original_df.merge(games_df.drop('Short_Timeouts', axis=1), on=['Game_ID', 'Season', 'Game_Date'])
print(combined.isnull().sum())
combined.head(10)

Season                0
Game_ID               0
Game_Date             0
Away_Team             0
Home_Team             0
Country               0
Rounded Viewers       0
Team                  0
Location              0
Wins_Entering_Gm      0
Losses_Entering_Gm    0
Team_Minutes          0
Final_Score           0
Lead_Changes          0
Ties                  0
Largest_Lead          0
Full_Timeouts         0
Qtr_4_Score           0
L2M_Score             0
dtype: int64


,Season,Game_ID,Game_Date,Away_Team,Home_Team,Country,Rounded Viewers,Team,Location,Wins_Entering_Gm,Losses_Entering_Gm,Team_Minutes,Final_Score,Lead_Changes,Ties,Largest_Lead,Full_Timeouts,Qtr_4_Score,L2M_Score
0,2016-17,21600001,10/25/2016,NYK,CLE,C113,18,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,82.0,113.0
1,2016-17,21600001,10/25/2016,NYK,CLE,C113,18,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,64.0,84.0
2,2016-17,21600001,10/25/2016,NYK,CLE,C193,1,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,82.0,113.0
3,2016-17,21600001,10/25/2016,NYK,CLE,C193,1,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,64.0,84.0
4,2016-17,21600001,10/25/2016,NYK,CLE,C126,4,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,82.0,113.0
5,2016-17,21600001,10/25/2016,NYK,CLE,C126,4,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,64.0,84.0
6,2016-17,21600001,10/25/2016,NYK,CLE,C163,11,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,82.0,113.0
7,2016-17,21600001,10/25/2016,NYK,CLE,C163,11,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,64.0,84.0
8,2016-17,21600001,10/25/2016,NYK,CLE,C73,3,CLE,H,0,0,240.0,117.0,1.0,3.0,32.0,4.0,82.0,113.0
9,2016-17,21600001,10/25/2016,NYK,CLE,C73,3,NYK,A,0,0,240.0,88.0,0.0,1.0,4.0,4.0,64.0,84.0


In [8]:
combined.shape

(561250, 19)

In [9]:
original_df.shape

(280625, 7)

In [10]:
games_df.shape

(4920, 16)

In [11]:
all_countries = ["C-%s" % e for e in original_df["Country"].unique().tolist()]
df_index = list(combined.columns) + all_countries
new_df = pd.DataFrame(index = df_index)

In [12]:
list(combined.columns)

['Season',
 'Game_ID',
 'Game_Date',
 'Away_Team',
 'Home_Team',
 'Country',
 'Rounded Viewers',
 'Team',
 'Location',
 'Wins_Entering_Gm',
 'Losses_Entering_Gm',
 'Team_Minutes',
 'Final_Score',
 'Lead_Changes',
 'Ties',
 'Largest_Lead',
 'Full_Timeouts',
 'Qtr_4_Score',
 'L2M_Score']

In [14]:
for game_id in combined["Game_ID"].unique():
    # Define the new series using the same index as the dataframe it will be concatenated to
    game_series = pd.Series(index=df_index)
    # Isolate the rows relevant for the current game
    game_rows = combined.loc[combined["Game_ID"] == game_id, :]

    # Since much of the data is repeated, get important information from the first row of the game
    game_series.loc[["Season", "Home_Team", "Away_Team", "Game_Date", 'Final_Score',
         'Lead_Changes',
         'Ties',
         'Largest_Lead',
         'Full_Timeouts',
         'Qtr_4_Score',
         'L2M_Score'
    ]] = game_rows.iloc[0, :].loc[["Season", "Home_Team", "Away_Team", "Game_Date", 'Final_Score',
         'Lead_Changes',
         'Ties',
         'Largest_Lead',
         'Full_Timeouts',
         'Qtr_4_Score',
         'L2M_Score'
    ]]

    # Add each country's viewership to the new series in the appropriate index location
    # NOTE: There is probably a more efficient way to do this
    for i,row in game_rows.iterrows():
        game_series.loc["C-%s" % row["Country"]] = row["Rounded Viewers"]

    # Add the game's data as a new column in the full df
    new_df[game_id] = game_series

# Currently the columns are game_ids and the rows are the information/data,
#   swap these to be more compatible with supervised learning methods using transpose
new_df = new_df.T

# Add the total viewers by summing over the country columns
country_cols = [e for e in new_df.columns if "C-" in e]
new_df["TotalViewers"] = new_df.loc[:, country_cols].sum(axis=1)

In [16]:
new_df = new_df.fillna(0)

In [24]:
# Converting teams to integer encoding
team_encoder = LabelEncoder()
team_encoder.fit(np.ravel([new_df["Home_Team"], new_df["Away_Team"]]))
new_df["Home_Team_Code"] = team_encoder.transform(new_df["Home_Team"])
new_df["Away_Team_Code"] = team_encoder.transform(new_df["Away_Team"])

vector_encoder = OneHotEncoder(sparse=False)
away_vectors = vector_encoder.fit_transform(new_df["Away_Team_Code"].values.reshape(-1, 1))
home_vectors = vector_encoder.fit_transform(new_df["Home_Team_Code"].values.reshape(-1, 1))
comb_vector = away_vectors + home_vectors
comb_df = pd.DataFrame(data=comb_vector, index=new_df.index, columns=["T-%s" % e for e in team_encoder.classes_])

merged_df = pd.concat([new_df, comb_df], axis=1)

In [25]:
# Adding day of week and indicator if that day is a weeknight (or not)
merged_df["DayOfWeek"] = pd.to_datetime(merged_df["Game_Date"]).dt.weekday_name
merged_df["Weeknight"] = merged_df["DayOfWeek"].apply(lambda cell: 1 if cell in
                ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday"] else 0)

In [29]:
# Outputting the revised data frame in a 'per game' format
merged_df.to_csv("training_set_games_complete.csv", index=True, index_label="Game_ID")

In [30]:
import pandas as pd
import numpy as np
import statsmodels.api as smi
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

# Read in file created from 'single_game_reformatting.py' script
input_filename = "training_set_games_complete.csv"
df = pd.read_csv(input_filename)

In [36]:
list(df.columns)

['Game_ID',
 'Season',
 'Game_ID.1',
 'Game_Date',
 'Away_Team',
 'Home_Team',
 'Country',
 'Rounded Viewers',
 'Team',
 'Location',
 'Wins_Entering_Gm',
 'Losses_Entering_Gm',
 'Team_Minutes',
 'Final_Score',
 'Lead_Changes',
 'Ties',
 'Largest_Lead',
 'Full_Timeouts',
 'Qtr_4_Score',
 'L2M_Score',
 'C-C113',
 'C-C193',
 'C-C126',
 'C-C163',
 'C-C73',
 'C-C128',
 'C-C208',
 'C-C67',
 'C-C177',
 'C-C171',
 'C-C176',
 'C-C124',
 'C-C189',
 'C-C224',
 'C-C51',
 'C-C42',
 'C-C212',
 'C-C201',
 'C-C199',
 'C-C166',
 'C-C46',
 'C-C89',
 'C-C139',
 'C-C14',
 'C-C172',
 'C-C183',
 'C-C99',
 'C-C91',
 'C-C59',
 'C-C125',
 'C-C169',
 'C-C195',
 'C-C38',
 'C-C116',
 'C-C62',
 'C-C101',
 'C-C55',
 'C-C190',
 'C-C145',
 'C-C50',
 'C-C35',
 'C-C71',
 'C-C182',
 'C-C44',
 'C-C122',
 'C-C31',
 'C-C222',
 'C-C94',
 'C-C213',
 'C-C184',
 'C-C109',
 'C-C223',
 'C-C7',
 'C-C65',
 'C-C78',
 'C-C174',
 'C-C13',
 'C-C15',
 'C-C3',
 'C-C123',
 'C-C108',
 'C-C12',
 'C-C37',
 'C-C179',
 'C-C135',
 'C-C21',
 'C

In [39]:
# Get the columns to be used as inputs (Team indicator columns)
team_cols = [e for e in df.columns if "T-" in e]
input_cols = team_cols + [
                 'Team_Minutes',
                 'Final_Score',
                 'Lead_Changes',
                 'Ties',
                 'Full_Timeouts',
                 'Qtr_4_Score', 
                  'L2M_Score', 
                  'Weeknight'
                ]

# Get data in supervised learning format f:X -> y
X = df.loc[:, input_cols]
y = df.loc[:, "TotalViewers"]

# Initialize dictionary for tracking accuracy measures, where Mean- is using the training set mean for predictions
accuracy_dict = {"MSE":[],
                 "Mean-MSE":[],
                 "MAPE":[],
                 "Mean-MAPE":[]}

In [40]:
# Perform 10-Fold Cross-Validation
kfold = KFold(n_splits=10, shuffle=True)
for train_indices, test_indices in kfold.split(X):
    # Get the training data and fit a LinearRegression to the training data
    X_train, y_train = X.loc[train_indices, :], y.loc[train_indices]
    fold_model = LinearRegression()
    fold_model.fit(X=X_train, y=y_train)

    # Make predictions on the test data from the trained model
    X_test, y_test = X.loc[test_indices, :], y.loc[test_indices]
    predictions = fold_model.predict(X=X_test)

    # Calculate the Mean Square Error of the current test fold predictions
    error = predictions - y_test
    mean_error = y_train.mean() - y_test

    fold_mse = np.mean(error**2)
    accuracy_dict["MSE"].append(fold_mse)

    # Calculate the Sum of Squared Error of the current test fold predictions
    fold_sse = np.mean(mean_error**2)
    accuracy_dict["Mean-MSE"].append(fold_sse)

    # Calculate the Mean Absolute Percentage Error (MAPE) used by NBA to grade submissions
    fold_mape = np.mean(np.abs(error)/y_test)
    accuracy_dict["MAPE"].append(fold_mape)

    fold_mean_mape = np.mean(np.abs(mean_error)/y_test)
    accuracy_dict["Mean-MAPE"].append(fold_mean_mape)
# Converting the accuracy dictionary to a DataFrame
#   with columns as the accuracy measures and index as the fold number
results_df = pd.DataFrame.from_dict(accuracy_dict, orient='columns')
results_df["R2"] = results_df.apply(lambda row: 1.0 - row["MSE"]/row["Mean-MSE"], axis=1)
results_df["MAPE-R2"] = results_df.apply(lambda row: 1.0 - row["MAPE"]/row["Mean-MAPE"], axis=1)
print(results_df)
print(results_df.mean(axis=0).round(3))

       MAPE           MSE  Mean-MAPE      Mean-MSE        R2   MAPE-R2
0  0.368074  3.724508e+07   0.641757  1.196008e+08  0.688588  0.426458
1  0.370433  5.814025e+07   0.721901  1.826332e+08  0.681656  0.486865
2  0.709384  1.485735e+08   0.569234  9.601719e+07 -0.547364 -0.246208
3  0.375186  4.979865e+07   0.622621  1.133968e+08  0.560846  0.397409
4  0.393123  4.807175e+07   0.799929  1.184461e+08  0.594147  0.508552
5  0.333912  4.773671e+07   0.768047  1.242597e+08  0.615831  0.565245
6  0.334505  5.432174e+07   0.675664  1.346216e+08  0.596486  0.504925
7  0.361056  4.419589e+07   0.773284  1.041794e+08  0.575771  0.533088
8  0.382072  4.019002e+07   0.766745  1.005354e+08  0.600240  0.501696
9  0.371152  3.908455e+07   0.712189  9.716804e+07  0.597763  0.478857
MAPE         4.000000e-01
MSE          5.673582e+07
Mean-MAPE    7.050000e-01
Mean-MSE     1.190858e+08
R2           4.960000e-01
MAPE-R2      4.160000e-01
dtype: float64


In [41]:
# Using Statsmodels for output on full set to see if the coefficients make sense
# (they appear to, as CLE and GSW are highest, MEM and PHX are lowest)
#    Having weeknight games, particularly on Tuesday also appears to drop the amount of viewers
# The explanatory R-Squared is 0.635 with Adj R-Squared of 0.628,
#   the closeness of these to the CV-R2 indicates there is not much overfitting occurring here

Xc = smi.add_constant(X)
ols = smi.OLS(endog=y, exog=Xc)
results = ols.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           TotalViewers   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     94.88
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        21:51:41   Log-Likelihood:                -20424.
No. Observations:                2000   AIC:                         4.092e+04
Df Residuals:                    1963   BIC:                         4.113e+04
Df Model:                          36                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          7952.6682   1471.059      5.406